In [1]:
if 'google.colab' in str(get_ipython()):
    # Colab specific setup
    assignment_path = '/content/gdrive/My Drive/CIS680/FinalProject/'
    
    # Mount your drive
    from google.colab import drive
    drive._mount("/content/gdrive")
    
    # Setup assignment folder and switch
    import os
    os.makedirs(assignment_path, exist_ok=True)
    os.chdir(assignment_path)

Mounted at /content/gdrive


# Imports

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import scipy
from scipy import spatial
import cv2

In [3]:
!pip install matplotlib opencv-python notebook tqdm
# !pip install torch==1.4.0 torchvision==0.5.0

# Get center of mass for RLE encoded mask

In [ ]:
import pycocotools.mask as mask
from shapely.geometry import Polygon
import pycocotools.mask as mask

def polygonCOMFromMask(compactRLESegmentation):
  maskedArr = mask.decode(compactRLESegmentation)
  area = float((maskedArr > 0.0).sum())
  # adapted from https://github.com/hazirbas/coco-json-converter/blob/master/generate_coco_json.py
  contours, _ = cv2.findContours(maskedArr, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  segmentation = []
  valid_poly = 0
  for contour in contours:
  # Valid polygons have >= 6 coordinates (3 points)
     if contour.size >= 6:
        segmentation.append(contour.astype(float).flatten().tolist())
        valid_poly += 1
  if valid_poly == 0:
     return -1
  contour = np.array(segmentation[0])
  contour = Polygon(contour.reshape((-1, 2)))
  com = list(list(contour.centroid.coords)[0])
  return com

# Convert annotations in the desired point annotation and boxes format

Provide the annotation file path for the machine parts dataset in `annotations_path` variable

In [ ]:
import json
annotations_path = "data/annotations.json"
with open(annotations_path) as f:
  data = json.load(f)
annotations = {}
id2file = {}
for image in data["images"]:
  id2file[image["id"]] = image["file_name"]
  annotations[image["file_name"]] = {"box_examples_coordinates": [], "points": []}
for annotation in data["annotations"]:
  x1, y1, x2, y2 = annotation["bbox"][0], annotation["bbox"][1], annotation["bbox"][0]+annotation["bbox"][2], annotation["bbox"][1]+annotation["bbox"][3]
  boxes = [[x1, y1], [x1, y2], [x2, y1], [x2, y2]]
  annotations[id2file[annotation["image_id"]]]["box_examples_coordinates"].append(boxes)
  com = polygonCOMFromMask(annotation["segmentation"])
  if com != -1:
    annotations[id2file[annotation["image_id"]]]["points"].append(com)

Provide where the converted annotations should be saved in the `points_annotation_path` variable

In [ ]:
points_annotation_path = "data/parts_new_annotations_v2.json"
with open(points_annotation_path, "w") as f:
  json.dump(annotations, f)

# Generate Density Maps

Specifiy where you want the density maps to be stored at `gt_dir`

In [39]:
gt_dir = "parts_gt_density_maps/"
def matlab_style_gauss2D(shape=(3,3),sigma=0.5):
    """
    2D gaussian mask - should give the same result as MATLAB's
    fspecial('gaussian',[shape],[sigma])
    http://stackoverflow.com/questions/17190649/how-to-obtain-a-gaussian-filter-in-python
    """
    m,n = [(ss-1.)/2. for ss in shape]
    y,x = np.ogrid[-m:m+1,-n:n+1]
    h = np.exp( -(x*x + y*y) / (2.*sigma*sigma) )
    h[ h < np.finfo(h.dtype).eps*h.max() ] = 0
    sumh = h.sum()
    if sumh != 0:
        h /= sumh
    return h

def generate_density_map(points, filename):
  points = np.array(points).astype(int)
  tree = scipy.spatial.KDTree(points.copy(), leafsize=10)
  dists, neighbours = tree.query(points, k = 2)
  avg = np.average(dists[:, 1])

  test = np.zeros((1080, 1080))
  for i in range(points.shape[0]):
      y = points[i, 1]
      x = points[i, 0]
      test[y, x] = 1

  filt = matlab_style_gauss2D((avg, avg), avg/4)
  gt_generated = cv2.filter2D(test.copy(), -1, filt, 0)
  gt_generated = cv2.resize(gt_generated, (384, 384))
  gt_generated = gt_generated * (7.91015625)
  # print(gt_generated.sum())
  # plt.imshow(gt_generated)
  with open(gt_dir+filename[:-4]+".npy", "wb") as f:
    np.save(f, gt_generated)

In [ ]:
with open("data/parts_new_annotations_v2.json") as f:
  data = json.load(f)
  for i, annotation in enumerate(data):
    print(i)
    generate_density_map(data[annotation]["points"], annotation)

In [ ]:
# Sanity Check code
# from PIL import Image
# import json
# gt_dir = "parts_gt_density_maps"
# im_id = "899_angle8_img.png"
# with open(points_annotation_path) as f:
#     annotations = json.load(f)
# anno = annotations[im_id]
# bboxes = anno['box_examples_coordinates']
# dots = np.array(anno['points'])
# print(len(dots))

# generate_density_map(dots, "random")
# density_path = gt_dir + '/' + im_id.split(".png")[0] + ".npy"
# density = np.load(density_path).astype('float32')    
# print(density.sum())

# Scale Images

Store source image directory in `source_img_dir` variable and destination image directory where scaled images have to be stored at `dest_img_dir`

In [ ]:
source_img_dir = "parts_dataset/"
dest_img_dir = "parts_dataset_resized/"
for img_name in os.listdir(source_img_dir):
  print(img_name)
  image = cv2.imread(source_img_dir+img_name)
  image = cv2.resize(image, (384, 384))
  cv2.imwrite(+img_name, image)

# Scale Annotations

Type the path where you'll have to save the scaled annotations in `scaled_points_annotation_path` variable

In [ ]:
import json
import numpy as np
scaled_points_annotation_path = "data/parts_new_annotations_scaled.json"
with open(points_annotation_path) as f:
  data = json.load(f)
for annotation in data:
  boxes =  data[annotation]["box_examples_coordinates"]
  points = data[annotation]["points"]
  boxes = np.array(boxes) * (384/1080)
  boxes = boxes.astype(int)
  points = np.array(points) * (384/1080)
  boxes, points = boxes.tolist(), points.tolist()
  data[annotation]["box_examples_coordinates"] = boxes
  data[annotation]["points"] = points
with open(scaled_points_annotation_path, "w") as f:
  json.dump(data, f)

# Generate dataset split

Store the dataset split json path at `dataset_split_path` and image classes text file path at `image_classes_txt_path` variable

In [ ]:
import random
dataset_split_path = "dataset_split.json"
images = os.listdir(dest_img_dir)
random.shuffle(images)
train = images[:6480]
val = images[6480:7290]
test = images[7290:]
data = {"train": train, "val": val, "test": test}
with open(dataset_split_path, "w") as f:
  json.dump(data, f)

In [ ]:
import random
image_classes_txt_path = "image_classes.txt"
images = os.listdir(dest_img_dir)
with open(image_classes_txt_path, "w") as f:
  for image in images:
    f.write(image+" machine-parts\n")

# Install dependencies and clone repo

In [ ]:
!git clone https://github.com/cvlab-stonybrook/LearningToCountEverything

Cloning into 'LearningToCountEverything'...
remote: Enumerating objects: 93, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 93 (delta 30), reused 42 (delta 27), pack-reused 43
Unpacking objects: 100% (93/93), done.


# Training

In [ ]:
!python LearningToCountEverything/train.py

Streaming output truncated to the last 5000 lines.
actual-predicted:   49.0,   40.3, error:    8.7. Current MAE:  3.32, RMSE:  4.34 Best VAL MAE:  2.60, RMSE:  3.65:  41% 2672/6480 [11:14<14:29,  4.38it/s]
actual-predicted:   26.0,   28.6, error:    2.6. Current MAE:  3.32, RMSE:  4.34 Best VAL MAE:  2.60, RMSE:  3.65:  41% 2673/6480 [11:14<19:31,  3.25it/s]
actual-predicted:   21.0,   26.3, error:    5.3. Current MAE:  3.32, RMSE:  4.34 Best VAL MAE:  2.60, RMSE:  3.65:  41% 2674/6480 [11:14<18:17,  3.47it/s]
actual-predicted:   20.0,   21.8, error:    1.8. Current MAE:  3.32, RMSE:  4.34 Best VAL MAE:  2.60, RMSE:  3.65:  41% 2675/6480 [11:14<16:46,  3.78it/s]
actual-predicted:   20.0,   24.7, error:    4.7. Current MAE:  3.32, RMSE:  4.34 Best VAL MAE:  2.60, RMSE:  3.65:  41% 2676/6480 [11:15<15:36,  4.06it/s]
actual-predicted:   35.0,   37.1, error:    2.1. Current MAE:  3.32, RMSE:  4.34 Best VAL MAE:  2.60, RMSE:  3.65:  41% 2677/6480 [11:15<14:41,  4.32it/s]
actual-predicted:  

# Visualization

In [4]:
cd LearningToCountEverything/

/content/gdrive/My Drive/CIS680/FinalProject/LearningToCountEverything


In [5]:
from model import CountRegressor, Resnet50FPN
from utils import MAPS, Scales, Transform, extract_features
from PIL import Image

In [34]:
import torch
import json
scaled_points_annotation_path = "data/parts_new_annotations_scaled.json"
im_id = "399_angle8_img.png" # Change image id here
dest_img_dir = "../parts_dataset_resized/"
gt_dir = "../parts_gt_density_maps/"
with open("../"+scaled_points_annotation_path) as f:
    annotations = json.load(f)
anno = annotations[im_id]
bboxes = anno['box_examples_coordinates']
dots = np.array(anno['points'])
resnet50_conv = Resnet50FPN()
resnet50_conv.cuda()
resnet50_conv.eval()
model_path = "../logs/FamNet_run_1.pth" # Change model path here
regressor = CountRegressor(6, pool='mean')
regressor.load_state_dict(torch.load(model_path))
regressor.cuda()
regressor.eval()
anno = annotations[im_id]
bboxes = anno['box_examples_coordinates']
dots = np.array(anno['points'])

rects = list()
for bbox in bboxes:
    x1, y1 = bbox[0][0], bbox[0][1]
    x2, y2 = bbox[2][0], bbox[2][1]
    rects.append([y1, x1, y2, x2])
image = Image.open('{}{}'.format(dest_img_dir, im_id))
image.load()
plt.imshow(image)
plt.title("Original Image - {}".format(im_id))
plt.savefig("original_image.png")

In [35]:
density_path = gt_dir + im_id.split(".png")[0] + ".npy"
density = np.load(density_path).astype('float32')   
gt_cnt = dots.shape[0]
plt.imshow(density)
plt.title("Ground Truth Density Map - Count = {}".format(gt_cnt))
plt.savefig("gt_density_image.png")

../parts_gt_density_maps/399_angle8_img.npy


In [36]:
sample = {'image': image, 'lines_boxes': rects}
sample = Transform(sample)
image, boxes = sample['image'], sample['boxes']
image = image.cuda()
boxes = boxes.cuda()
with torch.no_grad():
  features = extract_features(resnet50_conv, image.unsqueeze(0), boxes.unsqueeze(0), MAPS, Scales)
  output = regressor(features)
  plt.imshow(output.cpu().numpy().squeeze(0).squeeze(0))
  pred_cnt = output.sum().item()
  plt.title("Predicted Density Map - Count = {}".format(pred_cnt))
  err = abs(gt_cnt - pred_cnt)
  print("Actual: {}, Predicted: {}, Error: {}".format(gt_cnt, pred_cnt, err))
  plt.savefig("pred_density_image.png")

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3635: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


Actual: 20, Predicted: 19.1412410736084, Error: 0.8587589263916016
